# Unzip Dataset

In [ ]:
import os
root_dir = '/content/drive/MyDrive/SatriaData'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pwd

In [ ]:
# !rm

In [ ]:
os.chdir(root_dir)
!ls

 CNN_acc.png
 DataTest.csv
'Data Test for BDC 2023 - Penyisihan'
'Data Test for BDC 2023 - Penyisihan.zip'
 DataTrain
 DataTrain.csv
'Data Train for BDC 2023 - Penyisihan'
'Data Train for BDC 2023 - Penyisihan.zip'
 __MACOSX


In [ ]:
# !unzip 'Data Train for BDC 2023 - Penyisihan.zip' -d .
# !unzip 'Data Test for BDC 2023 - Penyisihan.zip' -d .

# # setelah ini, pindahkan file .csv di luar

In [ ]:
# !mv Data\ Train\ for\ BDC\ 2023\ -\ Penyisihan/DataTrain.csv .
# !mv Data\ Test\ for\ BDC\ 2023\ -\ Penyisihan/DataTest.csv .

In [ ]:
train_data_dir = os.path.join(root_dir, "Data Train for BDC 2023 - Penyisihan")
test_data_dir = os.path.join(root_dir, "Data Test for BDC 2023 - Penyisihan")

In [ ]:
# !ls

In [ ]:
import string
classes = list(string.ascii_uppercase)

for i in range(0, 10):
  classes.append(str(i))

os.mkdir("DataTrain")
# for c in classes:
#   os.mkdir(os.path.join('DataTrain', c))

# os.mkdir("DataTest")
# for c in classes:
#   os.mkdir(os.path.join('DataTest', c))

# Preprocess Data

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from collections import namedtuple
from skimage.filters import threshold_local
from skimage import segmentation
from skimage import measure
from imutils import perspective
import imutils

import string

In [ ]:
classes = list(string.ascii_uppercase)

for i in range(0, 10):
  classes.append(str(i))

numeric_classes = [i for i in range(len(classes))]

labeltoidx = dict(zip(classes, numeric_classes))
idxtolabel = dict(zip(numeric_classes, classes))

In [ ]:
df_train = pd.read_csv('DataTrain.csv', delimiter=";")
df_train.head()

,Unnamed: 0,Vehicleregistrationplate,NameofFile
0,0,A7814,DataTrain1.png
1,1,B1074QO,DataTrain2.png
2,2,B1031QO,DataTrain3.png
3,3,B187EDA,DataTrain4.png
4,4,B1089VD,DataTrain5.png


In [ ]:
df_test = pd.read_csv('DataTest.csv', delimiter=";")
df_test.head()

,Unnamed: 0,Name of File
0,0,DataTest1.png
1,1,DataTest2.png
2,2,DataTest3.png
3,3,DataTest4.png
4,4,DataTest5.png


In [ ]:
# !ls

In [ ]:
def preprocess_images_train(df):
  image_data = []
  label_data = []
  # image_data = np.array([])
  image_paths = df.NameofFile.values
  labels = df.Vehicleregistrationplate.values

  for label, image_file in zip(labels, image_paths):
    # Load gambar
    img = cv2.imread(os.path.join(train_data_dir, image_file))
    # Resize gambar
    r = 800.0 / img.shape[1]
    dim = (800, int(img.shape[0]*r))
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

    # mengubah gambar menjadi grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2RGB)

    # Noise Reduction
    gray = cv2.bilateralFilter(gray, 11, 17, 17)

    # edge detection
    edged = cv2.Canny(gray, 70, 200) #Perform Edge detection
    # edged = cv2.Sobel(gray, cv2.CV_64F, 1, 1, 5)

    # melakukan segmentasi menggunakan MSER
    mser = cv2.MSER_create()
    # regions, boundingBoxes = mser.detectRegions(gray)
    regions, boundingBoxes = mser.detectRegions(edged)
    boundingBoxes = np.unique(boundingBoxes, axis=0)

    candidates = []
    # candidates = np.array([])
    x_prev = 0

    for i, box in enumerate(boundingBoxes):
      x, y, w, h = box

      if x in range(x_prev+5):
        continue
      else:
        x_prev = x
        cropped = img[y:y+h, x:x+w]
        cy, cx, _ = cropped.shape
        if (40 < cx < 140) and (100 < cy < 200):
        # if (1.65 < cropped.shape[0]/cropped.shape[1] < 3.7):
          V = cropped
          T = threshold_local(V, 57, offset=15, method="gaussian")
          thresh = (V > T).astype("uint8") * 255
          thresh = cv2.bitwise_not(thresh)
          candidates.append(thresh)

          # candidates.append(cropped)

    label_char = [*label]

    # if len(label_char) == len(candidates):
    if len(label_char) > len(candidates):
      # print("found")
      for i, thimg in enumerate(candidates):
        cv2.imwrite(f"DataTrain/{image_file[:-4]}-{i}_{label_char[i]}.png", cv2.cvtColor(thimg, cv2.COLOR_BGR2GRAY))
        # cv2.imwrite(f"DataTrain/{label_char[i]}/{image_file[:-4]}-{i}.png", cv2.cvtColor(thimg, cv2.COLOR_BGR2GRAY))

        # Rotate and save the images
        angle = random.randint(0, 360)  # Random rotation angle
        (h, w) = thimg.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(thimg, M, (w, h))
        cv2.imwrite(f"DataTrain/{image_file[:-4]}-{i}_{label_char[i]}_rotated.png", rotated)

    # image_data.append(candidates)
  return image_data #[N, candidate]

In [ ]:
def preprocess_images_test(df):
  image_data = []
  label_data = []
  # image_data = np.array([])
  image_paths = df.NameofFile.values
  labels = df.Vehicleregistrationplate.values

  for label, image_file in zip(labels, image_paths):
    # Load gambar
    img = cv2.imread(os.path.join(train_data_dir, image_file))
    # Resize gambar
    r = 800.0 / img.shape[1]
    dim = (800, int(img.shape[0]*r))
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

    # mengubah gambar menjadi grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2RGB)

    # Noise Reduction
    gray = cv2.bilateralFilter(gray, 11, 17, 17)

    # edge detection
    edged = cv2.Canny(gray, 70, 200) #Perform Edge detection
    # edged = cv2.Sobel(gray, cv2.CV_64F, 1, 1, 5)

    # melakukan segmentasi menggunakan MSER
    mser = cv2.MSER_create()
    # regions, boundingBoxes = mser.detectRegions(gray)
    regions, boundingBoxes = mser.detectRegions(edged)
    boundingBoxes = np.unique(boundingBoxes, axis=0)

    candidates = []
    # candidates = np.array([])
    x_prev = 0

    for i, box in enumerate(boundingBoxes):
      x, y, w, h = box

      if x in range(x_prev+5):
        continue
      else:
        x_prev = x
        cropped = img[y:y+h, x:x+w]
        cy, cx, _ = cropped.shape
        if (40 < cx < 140) and (100 < cy < 200):
        # if (1.65 < cropped.shape[0]/cropped.shape[1] < 3.7):
          V = cropped
          T = threshold_local(V, 57, offset=15, method="gaussian")
          thresh = (V > T).astype("uint8") * 255
          thresh = cv2.bitwise_not(thresh)
          candidates.append(thresh)

          # candidates.append(cropped)

    label_char = [*label]

    # if len(label_char) == len(candidates):
    if len(label_char) > len(candidates):
      # print("found")
      for i, thimg in enumerate(candidates):
        cv2.imwrite(f"DataTrain/{image_file[:-4]}-{i}_{label_char[i]}.png", cv2.cvtColor(thimg, cv2.COLOR_BGR2GRAY))
        # cv2.imwrite(f"DataTest/{label_char[i]}/{image_file[:-4]}-{i}.png", cv2.cvtColor(thimg, cv2.COLOR_BGR2GRAY))

    # image_data.append(candidates)
  return image_data #[N, candidate]

In [ ]:
# # img_pth = os.listdir(train_data_dir)[25]
# # images = preprocess_images([img_pth])

# # results = preprocess_images(df_train.iloc[:100])
results = preprocess_images_train(df_train)

/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)


In [ ]:
# results = preprocess_images_test(df_test)

In [ ]:
# for c in classes:
#   print(len(os.listdir(f"DataTrain/{c}")))

# Modeling

In [ ]:
import os
import time

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets, models

In [ ]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((255,255)),
    transforms.RandomRotation(45),
    transforms.RandomPerspective(0.5, 0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.3)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
    ])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((255,255)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
    ])

In [ ]:
# class CharDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset, transform=None):
#         self.dataset = dataset
#         self.transform = transform

#     def __getitem__(self, index):
#         x = dataset[index][0] if not self.transform else self.transform(dataset[index][0])
#         y = dataset[index][1]
#         return x, y

#     def __len__(self):
#         return len(dataset)

class ImageDataset(torch.utils.data.Dataset):
  def __init__(self,img_folder,transform):
    self.transform=transform
    self.img_folder=img_folder

    self.image_names = os.listdir(img_folder) # "DataTrain/{image_file[:-4]}-{i}_{label_char[i]}.png"

    # self.labels = np.array(self.csv.drop(['Id', 'Genre'], axis=1))
    self.labels = np.array([x[-5] for x in self.image_names])

#The __len__ function returns the number of samples in our dataset.
  def __len__(self):
    return len(self.image_names)

  def __getitem__(self,index):
    # image=cv2.imread(self.img_folder+self.image_names[index]+'.png')
    image=cv2.imread(self.img_folder + "/" + self.image_names[index])
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

    image=self.transform(image)
    targets=self.labels[index]

    sample = {'image': image,'labels':targets}

    return sample

In [ ]:
data_dir = 'DataTrain'
dataset = ImageDataset(data_dir, train_transforms)
train_loader = torch.utils.data.DataLoader(dataset , batch_size=8, shuffle=False,
                               num_workers=1, drop_last=True)

In [ ]:
sample = next(iter(train_loader))
print(sample['image'].shape)

torch.Size([8, 3, 255, 255])


In [ ]:
num_classes = len(list(set(dataset.labels)))
print(num_classes)

24


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(12544, 1024)
        self.fc2 = nn.Linear(1024, 36)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
model_scratch = Model()

In [ ]:
def viz(model_name, train_acc, val_acc, train_loss, val_loss):
    epoch = range(1, len(train_acc) + 1)

    plt.plot(epoch, train_acc, 'r', label='Training Acc')
    plt.plot(epoch, val_acc, 'b', label='Validation Acc')
    plt.title(model_name + ' training and validation acc')
    plt.legend()
    plt.savefig(model_name + '_acc.png')

    plt.plot(epoch, train_loss, 'r', label='Training Loss')
    plt.plot(epoch, val_loss, 'b', label='Validation Loss')
    plt.title(model_name + ' training and validation loss')
    plt.legend()
    # plt.savefig(model_name + '_loss.png')

In [ ]:
# def training_process(model, n_epochs, train_loader, val_loader):
def training_process(model, n_epochs, train_loader):
    # model_name = model
    # model = model_list[model]

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    print('Training on {} ...'.format(device))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

    val_loss_min = np.Inf  # track change in validation loss
    hist_train_acc, hist_val_acc, hist_train_loss, hist_val_loss = [], [], [], []

    for epoch in range(1, n_epochs + 1):
        t0 = time.time()
        t_running_loss, v_running_loss, t_correct, t_total, v_correct, v_total = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

        model.train()
        # for data, target in train_loader:
        for trainingdata in train_loader:
            # data, target = data.to(device), target.to(device)
            data, target = trainingdata['image'], trainingdata['labels']
            data = data.to(device)
            target = [labeltoidx[t] for t in target]

            target = torch.from_numpy(np.array(target)).to(device)

            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            t_running_loss += loss.item()
            _, predicted = output.max(1)
            t_total += target.size(0)
            t_correct += predicted.eq(target).sum().item()

        train_acc = t_correct / t_total
        train_loss = t_running_loss / len(train_loader)

        # model.eval()
        # for data, target in val_loader:
        #     data, target = data.to(device), target.to(device)
        #     output = model(data)
        #     loss = criterion(output, target)

        #     v_running_loss += loss.item()
        #     _, predicted = output.max(1)
        #     v_total += target.size(0)
        #     v_correct += predicted.eq(target).sum().item()

        # val_acc = v_correct / v_total
        # val_loss = v_running_loss / len(val_loader)

        # hist_train_acc.append(train_acc)
        # hist_val_acc.append(val_acc)
        # hist_train_loss.append(train_loss)
        # hist_val_loss.append(val_loss)

        t1 = time.time() - t0
        # print(
        #     'Epoch: {} \tTrain Acc: {}/{}({:.2f}) \tVal Acc: {}/{}({:.2f}) \tTrain Loss: {:.4f} \tVal Loss: {:.4f} \tTraining time: {} seconds'.format(
        #         epoch, t_correct, t_total, train_acc, v_correct, v_total, val_acc, train_loss, val_loss, t1))
        print(
            'Epoch: {} \tTrain Acc: {}/{}({:.2f}) \tTrain Loss: {:.4f} \tTraining time: {} seconds'.format(
                epoch, t_correct, t_total, train_acc, train_loss, t1))

        # save model if validation loss has decreased
        # if val_loss <= val_loss_min:
        #     print('Validation loss decreased ({:.2f} --> {:.2f}).  Saving model ...'.format(val_loss_min, val_loss))
        #     torch.save(model.state_dict(), model_name+'.pth')
        #     val_loss_min = val_loss

    # viz("CNN", hist_train_acc, hist_val_acc, hist_train_loss, hist_val_loss)

training_process(model_scratch, 100, train_loader)

Training on cuda ...


KeyError: ignored

# Testing

In [ ]:
def preprocess_single(image_path):
  img = cv2.imread(image_path)

  # Resize gambar
  r = 800.0 / img.shape[1]
  dim = (800, int(img.shape[0]*r))
  img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

  # mengubah gambar menjadi grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.cvtColor(gray, cv2.COLOR_BGR2RGB)

  # Noise Reduction
  gray = cv2.bilateralFilter(gray, 11, 17, 17)

  # edge detection
  edged = cv2.Canny(gray, 70, 200) #Perform Edge detection

  # melakukan segmentasi menggunakan MSER
  mser = cv2.MSER_create()
  # regions, boundingBoxes = mser.detectRegions(gray)
  regions, boundingBoxes = mser.detectRegions(edged)
  boundingBoxes = np.unique(boundingBoxes, axis=0)

  # candidates = np.array([])
  candidates = []
  x_prev = 0

  # imgboxed = img.copy()
  for i, box in enumerate(boundingBoxes):
    x, y, w, h = box
    # cv2.rectangle(imgboxed, (x, y), (x + w, y + h), (255,0,0), 4)
    if x in range(x_prev+5):
      continue
    else:
      x_prev = x
      cropped = img[y:y+h, x:x+w]
      cy, cx, _ = cropped.shape
      if (40 < cx < 140) and (100 < cy < 200):
        V = cropped
        T = threshold_local(V, 57, offset=15, method="gaussian")
        thresh = (V > T).astype("uint8") * 255
        thresh = cv2.bitwise_not(thresh)
        candidates.append(thresh)
  # plt.imshow(imgboxed)
  return candidates

In [ ]:
df_test = pd.read_csv("DataTest.csv", delimiter=';')
df_test.head()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

preds = []
for test_file in df_test['Name of File'].values:
  test_image_path = os.path.join(test_data_dir, test_file)
  plate_numbers = preprocess_single(test_image_path)
  if len(plate_numbers) == 0:
    preds.append('')
  else:
    plate_num = []
    with torch.no_grad():
      for char_img in plate_numbers:
        out = model_scratch(test_transforms(char_img).unsqueeze(0).to(device))
        plate_num.append(torch.argmax(out, 1).cpu().detach().numpy()[0])
    preds.append(plate_num)

In [ ]:
nomor_plat = [1, 27, 1, 1, 1]
nomor_plat = [idxtolabel[no] for no in nomor_plat]
''.join(nomor_plat)

In [ ]:
predicted = []
for pred in preds:
  if pred == '' :
    continue
  else :
    plate_number = [idxtolabel[num] for num in pred]
    predicted.append(''.join(plate_number))

predicted